In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dateutil.parser import parse
from collections import Counter
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split, cross_val_score,cross_val_predict
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)

Pre-Processing

In [ ]:
data = pd.read_csv('books_1.Best_Books_Ever.csv') #read the data
df = pd.DataFrame(data) #create the dataframe
df = df.dropna(subset=['description','language','genres','pages','publishDate','ratingsByStars'])  #drop the null values of the needed columns from the dataframe

df['ratingsByStars'] = df['ratingsByStars'].str.strip('[]') #remvove [] brackets
df['ratingsByStars'] = df['ratingsByStars'].str.replace("'",'') #remove the ' from each word
df[['ratingStar5','ratingStar4','ratingStar3','ratingStar2','ratingStar1']] = df['ratingsByStars'].str.split(',',expand=True) #split the string by each comma and add the new
#string to each of the new columns created

df['genres'] = df['genres'].str.strip('[]') #remove [] brackets
df['genreSingle'] = df['genres'].str.split(',').str[0].str.strip() #extract only the first string from the stri split method
df['Year'] = pd.to_datetime(df['publishDate'], errors='coerce') #conver the column to datetime object and if we encounter error set the value to null

df = df[df['publishDate'].str.contains(r'\d')] #records that have non-numeric value in publish date remove them from the dataframe

df['Year'] = df['Year'].dt.year #extract the year from the dates

Questions for extracting data (5 in total)

In [ ]:
#1st graph (1.1 Question)
plt.hist(df['rating'],color='red')  #show the rating column of dataframe in a histogram
plt.title('Histogram for Ratings')
plt.xlabel('Rating')
plt.ylabel('Occurances')
plt.show()

#2nd graph (1.2 Question)
df['pages'] = df['pages'].astype(str) #in order to clean the page column that contains strings (18 cases found) first transform the object from int32(??) to str
df = df[df['pages'].str.isdigit()] # clean the dataframe for column page (drop non-numeric values)
df['pages'] = df['pages'].astype(int)   #cast to sort the dataframe treating pages as integers and not strings
dfSortedByPages = df.sort_values('pages', ascending=False,) #sort by descending order the dataframe by the pages column
df_top10 = dfSortedByPages.head(10)   #get only the top 10 cases
plt.barh(df_top10['title'], df_top10['pages'], color='blue') #create the bar-chart,on x-axis the number of pages and in y axis the title of the book

plt.xlabel('Pages') #add labels & title
plt.ylabel('Title')
plt.title('Top 10 books with most pages')
plt.show()

#3rd graph (1.3 Question)
df['ratingStar5'] = df['ratingStar5'].replace(r'^\s*$', 0, regex=True) #fill the whitespace values of column 'ratingStar5' with a zero digit so to be able to cast to int the column
df['ratingStar5'] = df['ratingStar5'].astype(int)   #cast to sort the dataframe treating 'ratingStar5' column as integers and not strings
dfNew = df[df['ratingStar5'] > 1000]  #get only the entries with 10.000 and above 5-star ratings first
dfSortedByPages = dfNew.sort_values('ratingStar5', ascending=False,) #sort by descending order the dataframe by the pages column
df_top10 = dfSortedByPages.head(10)   #get only the top 10 cases
plt.barh(df_top10['title'], df_top10['ratingStar5'], color='grey') #create the bar-chart,on x-axis the number of 5-star ratings and in y axis the title of the book


plt.xlabel('5-star ratings (in millions)') #add labels & title
plt.ylabel('Title')
plt.title('Top 10 books with most 5-star ratings')
plt.show()

#4rd graph (1.5 Question)
dfNew1 = df['author'].value_counts()  #distinct each author of the dataframe how many books has written
dfNew1 = dfNew1.head(10)   #get only the 10 top frequent authors of the dataframe
plt.barh(dfNew1.index,dfNew1.values, color='green')

plt.xlabel('Number of books written') #add labels & title
plt.ylabel('Author')
plt.title('Top 10 authors with most written books')
plt.show()

#5rd graph (1.8 Question)
dfNew2 = df['language'].value_counts()  #distinct each language of the dataframe how many books are written in each
dfNew2 = dfNew2.head(10)   #get only the 10 top most common languages of the dataframe
plt.barh(dfNew2.index,dfNew2.values, color='black')

plt.xlabel('Number') #add labels & title
plt.ylabel('Language')
plt.title('Books found in each language')
plt.show()

Recommendation system

In [ ]:
df = df[df['language'] == 'English'] #omit the books that are not written in English
vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words='english') #create a TfidfVectorizer object to extract unigrams and bigrams without stopwords of english language
tfidf = vectorizer.fit_transform(df['description']) #transform text to numbers (fitting of vectorizer)

def recommend(item_id, num):
      similar_rows = {}   #intializing the empty directory
      idx = df[df['bookId'] == item_id].index[0]     #find the index of the given Book Id in the dataframe to be able to get its similar books
      cos_sim = cosine_similarity(tfidf[idx],tfidf) #calculate the similarity of each row of a DataFrame and every other row
      similar_indices = cos_sim.argsort()[0][-num-1:-1][::-1] #sort the cosine similarity scores in ascending order and get last n values(since we need the most similar) omiting the first one that is itself
      print("Recommending ", num," books similar to: ", df.iloc[idx]['title'], 
            "\n---------------------------------------------------------\n")
      for index in similar_indices:
            similar_rows[df.iloc[index]['title']] = [df.iloc[index]['description'], cos_sim[0][index]]    #store the n-similar books in a dictionary with the needing info
            print("Recommended: ", df.iloc[index]['title'],           #print it in the format that is needed the results
                  "\nDescription: ", df.iloc[index]['description'],
                  "\n( score:", cos_sim[0][index], ")\n\n")

#for bookId in df['bookId']:
#      recommend(bookId,100) #example

Classification

In [ ]:
common_words = Counter(df['genreSingle']) #count how many books correspond to each genre

most_common = common_words.most_common(10) #keep the 10 most common

most_common_genres = [] #the most common method returns a tuple containing the genre name and
for word,count in most_common: #the number of the occurences for each genre in the dataframe
    most_common_genres.append(word) #so we split the tuple and keep only the genre name

new_df = df.copy() #create a copy of the dataframe
new_df = new_df[new_df['genreSingle'].apply(lambda x: any(word in x for word in most_common_genres))]
#apply a lamdba function to keep only the books that are in the 10 most common genres

columns_keep = ['genreSingle','description','bookId'] #we want to keep only these 3 columns
new_df = new_df.drop(columns=[col for col in new_df.columns if col not in columns_keep]) #delete the columns which are not in the list above

new_df['description'] = new_df['description'].str.lower() #make all words lower case
stop  = stopwords.words('english') #initialize stop words to english language
pat = r'\b(?:{})\b'.format('|'.join(stop)) #regex expression
new_df['description'] = new_df['description'].str.replace(pat,'',regex=True) #apply regex expr
new_df['description'] = new_df['description'].str.replace(r'\s+',' ',regex=True)
cleaned = []
elements = list(new_df['description']) 
for i in elements : #iterate over all elements in the description column
    FilteredText = re.sub('https?://[A-Za-z0-9./]+','',i) #clean also the links from each element of the column
    FilteredText = re.sub("[^a-zA-Z0-9]", " ",FilteredText) #clean also punctuation
    cleaned.append(re.sub(r'^RT[\s]+', '', FilteredText))
new_df['description'] = cleaned
new_df['description'] = new_df['description'].apply(lambda x: x.split()) #split the text in a list of lists
model = Word2Vec(new_df['description'],vector_size=200,window=5,min_count=2,sg=1,hs=0,negative=10,workers=4,seed=34) #initialize word2vec model
model.train(new_df['description'],total_examples=len(new_df),epochs=20) #train it

Save Files

In [ ]:
with open('W2V.pkl' ,"wb") as file: #open it
    pickle.dump(model,file) #save the model
with open('df.pkl' ,"wb") as file: #open it
    pickle.dump(new_df,file) #save the model

Open Files

In [3]:
with open('df.pkl',"rb") as file:
    new_df = pickle.load(file) #open dataframe
with open('W2V.pkl',"rb") as file:
    model = pickle.load(file) #open dataframe

Word Embeddings for classification

In [4]:
X = []
for document in new_df['description']:
    embeddings = [model.wv[word] for word in document if word in model.wv]
    if embeddings:
        document_embedding = np.mean(embeddings, axis=0)  # Average the word embeddings
    else:
        document_embedding = np.zeros(model.vector_size)  # Use zero vector for out-of-vocabulary words
    X.append(document_embedding)


Train Test Splitting

In [5]:
X = np.array(X)
y = new_df['genreSingle'] #split data to train test split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

Naive Bayes

In [15]:
clf = GaussianNB() #Naive Bayes method
clf.fit(X_train,y_train)
#y_pred = clf.predict(X_test)

# 10-fold cross-validation with metrics for GaussianNB method
kfold = KFold(n_splits=10,shuffle=True,random_state=42)
scores = cross_val_score(clf,X_test,y_test,cv=kfold)
print("Accuracy:", scores.mean())

y_pred = cross_val_predict(clf,X_test,y_test,cv=10)
report = classification_report(y_test,y_pred,output_dict=True,zero_division=0)

precision = report['macro avg']['precision']
recall = report['macro avg']['recall']
f_measure = report['macro avg']['f1-score']

print("Precision:", precision)
print("Recall:", recall)
print("F-measure:", f_measure)




Accuracy: 0.32292389629019214


/home/natedrake7/.local/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


Precision: 0.12012590431124553
Recall: 0.12243266161922085
F-measure: 0.10848565583662971


Support Vector Machines

In [26]:
param_grid = {'C': [0.1, 1, 10], 'gamma': [0.001, 0.01, 0.1]}

svm = SVC()
grid_search = GridSearchCV(svm, param_grid, scoring='accuracy',n_jobs=-1)
grid_search.fit(X_train,y_train)
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(best_params)
print(best_score)

/home/natedrake7/.local/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Random Forest

In [23]:
random_forest = RandomForestClassifier(n_jobs=-1,n_estimators=250,max_depth=256)
#random_forest.fit(X_train,y_train)
#y_pred = random_forest.predict(X_test)

# 10-fold cross validation with metrics for Random Forest method
kfold = KFold(n_splits=10,shuffle=True,random_state=42)
scores = cross_val_score(random_forest,X_test,y_test,cv=kfold)
print("Accuracy:", scores.mean())

y_pred = cross_val_predict(clf,X_test,y_test,cv=10)
report = classification_report(y_test,y_pred,output_dict=True,zero_division=0)

precision = report['macro avg']['precision']
recall = report['macro avg']['recall']
f_measure = report['macro avg']['f1-score']

print("Precision:", precision)
print("Recall:", recall)
print("F-measure:", f_measure)

Accuracy: 0.36762303712859956


/home/natedrake7/.local/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


Precision: 0.12012590431124553
Recall: 0.12243266161922085
F-measure: 0.10848565583662971
